In [1]:
import json
import os
import math
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import tensorflow as tf

C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Alexander\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

In [2]:
#Variable definitions
#signal-Signal S
#frame_size -divided frames
#hop_length-size intersection between frames
#P- given parameter
#defined function- Input = signal; Output = t_new point   (Step 1-4 in the Paper)
def amplitude_envelope(signal,sample_duration, frame_size=1024, hop_length=512,P=0.35):
    """Calculate the amplitude envelope of a signal with a given frame size nad hop length."""
    amplitude_envelope = []  #Define amplitude envolope list
    index_signal_amplitude_envelope=[]  #Define index for amplitude envelope
    # calculate amplitude envelope for each frame
    for i in range(0, len(signal), hop_length): 
        amplitude_envelope_current_frame = max(signal[i:i+frame_size]) 
        amplitude_envelope.append(amplitude_envelope_current_frame)
        #append index for each amplitude envelope obtained in each frame
        index_signal_amplitude_envelope.append(i+signal[i:i+frame_size].tolist().index(max(signal[i:i+frame_size].tolist())))
    
    ae_signal=np.array(amplitude_envelope)                        #list of amplitude envelope
    t=np.array(index_signal_amplitude_envelope)*sample_duration   #corresponding time for amplitude envelope
    t_max=t[ae_signal.tolist().index(max(ae_signal))]       #t_new=time for max amplitude - 0.35   #0.35 is a parameter
    
    if t_max<=P:
        t_new=0
    else:
        t_new=t_max-P
    return t_new

In [6]:
def save_mfcc(dataset_path, n_fft, hop_length):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.
        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "labels": [],  # 1,2,3,4
        "mfcc": [],    # STFTs...,mels,.....MFCCs
        "gender":[],   #male, female
        "speaker":[]   #names
    }
   
    time_o=[]
    #samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    #num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)
    j=0
    a=0
    p=0
    # loop through all sub-folder
    #Section 4.1.1 Signal length module
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        
        if dirpath is not dataset_path:
            # save gender label  in the mapping 
            gender_label = dirpath.split("_")[2]
            print("\nProcessing: {}".format(gender_label))
           
            # process all audio files 
            
            for f in filenames:
                # save DIGIT label  in the mapping
                digit_label = f.split("_")[0]
                # save speaker label  in the mapping
                speaker_label = f.split("_")[1]
                # Obtain digit index label in the mapping
                index_label = f.split("_")[2]
                # Save file path with source dir path
                file_path = os.path.join(dirpath, f)
                # extract speaker name from speaker folder name 
                name=dirpath.split("\\")[1].split("_")[0]
                
                # extract speaker index no from speaker folder name
                no=dirpath.split("\\")[1].split("_")[1]
                
                # input file wav file to obtain signal and new sample rate
                signal, sr = librosa.load(file_path, sr=SAMPLE_RATE) 
                
                # New output file name
                trimmed_path="recordings-github\\"+dirpath.split("\\")[1]+"\\"+digit_label+"_"+name+no+"_"+index_label+".wav"
                
                # Display
                print(trimmed_path)
                # Calculate duration in seconds for the input Signal
                dur=len(signal)/SAMPLE_RATE
                
                sample_duration = 1 / sr
                #
                if dur>=0.74:
                   # print(dur)
                    p+=1
                
                # Step 1-5
                if len(signal)>= num_expected_samples:  #Step 5 checking for trimming
                    t_new=amplitude_envelope(signal,sample_duration)  #Step 1-4
                    trim_signal, sr = librosa.load(file_path,sr=SAMPLE_RATE,offset=t_new,duration=DURATION) #trimming
                    padded_signal=trim_signal
                    if len(trim_signal)<num_expected_samples:
                        num_missing_samples = num_expected_samples - len(trim_signal)
                        padded_signal = np.pad(trim_signal,
                                              (0, num_missing_samples),
                                              mode="constant")      #Padding
                        a+=1
                elif len(signal) < num_expected_samples:        #Step 5 checking for padding
                    num_missing_samples = num_expected_samples - len(signal)
                    padded_signal = np.pad(signal,
                                          (0, num_missing_samples),
                                          mode="constant")       #Padding
                    
                #librosa.output.write_wav(trimmed_path, padded_signal, sr)
                
                # Section 4.1.2 Feature extraction module
                # Input new signal ; Output MFCCs
                # New signal=padded_signal; sr=sample rate; n_mfcc=no of mfcc coefficients; n_fft=interval for fft; hop_length=sliding window
                mfcc = librosa.feature.mfcc(padded_signal, sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                
                # Display mfcc dimension 
                print(mfcc.shape)
                
                # normalizing
                S_min=np.min(mfcc)
                S_max=np.max(mfcc)
                X_norm_mfcc=(mfcc - S_min)/(S_max - S_min)
                
                # append normalized mfccs features, labels, gender and speaker labels to each data containers
                data["mfcc"].append(X_norm_mfcc.T.tolist())
                data["labels"].append(digit_label)
                data["gender"].append(gender_label)
                data["speaker"].append(speaker_label)
                
                j+=1  
    
    # save MFCCs and other labels to json file
    with open(JSON_PATH, "w") as fp:
        json.dump(data, fp, indent=4)

In [9]:
# Defined parameters
DATASET_PATH = "recordings"   # source folder
JSON_PATH =  "MNIST-digit-sw-mfcc-norm-16.json"  #destination json file name
SAMPLE_RATE = 22050 #22050 #   8000
n_fft=512      #511  358   512          64
hop_length=256   #123  296   256       512
DURATION=0.74   #2   0.74
num_expected_samples=int(DURATION*SAMPLE_RATE)
num_mfcc=16

In [10]:
if __name__ == "__main__":
    save_mfcc(DATASET_PATH,n_fft, hop_length) # run save_mfcc function


Processing: male
recordings-github\speaker_10_male_uknown\0_speaker10_0.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_1.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_2.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_3.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_4.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_5.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_6.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_7.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_8.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\0_speaker10_9.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\1_speaker10_0.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\1_speaker10_1.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\1_speaker10_2.wav.wav
(16, 64)
recordings-github\speaker_10_male_uknown\1_spe

(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_0.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_1.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_2.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_3.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_4.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_5.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_6.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_7.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_8.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\2_speaker11_9.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\3_speaker11_0.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\3_speaker11_1.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\3_speaker11_2.wav.wav
(16, 64)
recordings-github\speaker_11_male_uknown\3_speaker11_3.

(16, 64)
recordings-github\speaker_12_male_uknown\3_speaker12_5.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\3_speaker12_6.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\3_speaker12_7.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\3_speaker12_8.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\3_speaker12_9.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_0.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_1.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_2.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_3.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_4.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_5.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_6.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_7.wav.wav
(16, 64)
recordings-github\speaker_12_male_uknown\4_speaker12_8.

(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_2.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_3.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_4.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_5.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_6.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_7.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_8.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\5_speaker13_9.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\6_speaker13_0.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\6_speaker13_1.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\6_speaker13_2.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\6_speaker13_3.wav.wav
(16, 64)
recordings-github\speaker_13_female_uknown\6_speaker13_4.wav.wav
(16, 64)
recordings-github\speaker_13_

(16, 64)
recordings-github\speaker_14_female_uknown\6_speaker14_4.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\6_speaker14_5.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\6_speaker14_6.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\6_speaker14_7.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\6_speaker14_8.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\6_speaker14_9.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\7_speaker14_0.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\7_speaker14_1.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\7_speaker14_2.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\7_speaker14_3.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\7_speaker14_4.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\7_speaker14_5.wav.wav
(16, 64)
recordings-github\speaker_14_female_uknown\7_speaker14_6.wav.wav
(16, 64)
recordings-github\speaker_14_

...

(16, 64)
recordings-github\speaker_4_male_uknown\1_speaker4_5.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\1_speaker4_6.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\1_speaker4_7.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\1_speaker4_8.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\1_speaker4_9.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_0.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_1.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_2.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_3.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_4.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_5.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_6.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_7.wav.wav
(16, 64)
recordings-github\speaker_4_male_uknown\2_speaker4_8.wav.wav
(16, 64)
recordings-

(16, 64)
recordings-github\speaker_5_male_uknown\3_speaker5_4.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\3_speaker5_5.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\3_speaker5_6.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\3_speaker5_7.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\3_speaker5_8.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\3_speaker5_9.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_0.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_1.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_2.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_3.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_4.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_5.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_6.wav.wav
(16, 64)
recordings-github\speaker_5_male_uknown\4_speaker5_7.wav.wav
(16, 64)
recordings-

(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_2.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_3.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_4.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_5.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_6.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_7.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_8.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\5_speaker6_9.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\6_speaker6_0.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\6_speaker6_1.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\6_speaker6_2.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\6_speaker6_3.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\6_speaker6_4.wav.wav
(16, 64)
recordings-github\speaker_6_female_uknown\6_speaker6_5.

(16, 64)
recordings-github\speaker_7_female_uknown\6_speaker7_8.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\6_speaker7_9.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_0.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_1.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_2.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_3.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_4.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_5.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_6.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_7.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_8.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\7_speaker7_9.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\8_speaker7_0.wav.wav
(16, 64)
recordings-github\speaker_7_female_uknown\8_speaker7_1.

(16, 64)
recordings-github\speaker_8_male_uknown\8_speaker8_7.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\8_speaker8_8.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\8_speaker8_9.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_0.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_1.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_2.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_3.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_4.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_5.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_6.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_7.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_8.wav.wav
(16, 64)
recordings-github\speaker_8_male_uknown\9_speaker8_9.wav.wav
(16, 64)

Processing: female
recordings-github\speaker_9_female_uknown\0_speaker9_0.wav.wa